Now, coming to Week 2 task. For each of the authors, you ideally want to know more about their background, topics of interest, twitter links etc.

One easy way to grab it is going their muckrack journalist profile page and web scrapping it from there (refer to chapter 2, beautifulsoup portion to know how to do it).

Two ways to do it: first, you can search through the muckrack sitemap CSV file for near duplicate matches in similar or same names and scrape their profiles. 

Second method is letting Bing/Google do the job aka take the author name and outlet name and search Google programmatically using Specrom APIs. For example if you want to know more about Dave Sebastian WSJ.com just use it as a search key and find muckrack profile and proceed from there. 

How about you trying doing do both approaches on top 10-30 authors and see which one works best?

https://algorithmia.com/algorithms/specrom/Google_scraper

https://algorithmia.com/algorithms/specrom/Bing_scraper

In [2]:
import pickle
import pandas as pd
import re
import Algorithmia
import bs4 as bs
import lxml
from urllib.request import Request, urlopen
from selenium import webdriver
import time

In [4]:
with open('../Data/news.pkl', 'rb') as f:
    news = pickle.load(f)

muckrack = pd.read_csv('../Data/muckrack_persons_fetchlist.csv')

In [5]:
muckrack

,Unnamed: 0,last_modified,url
0,0,2020-01-25,https://muckrack.com/__jackguy
1,1,2020-05-08,https://muckrack.com/__jbernstein
2,2,2019-04-18,https://muckrack.com/__jessclark
3,3,2020-08-12,https://muckrack.com/__jevans__
4,4,2020-05-28,https://muckrack.com/__katieoconnor
...,...,...,...
231727,231727,2018-12-02,https://muckrack.com/zuzana-diamant
231728,231728,2020-06-23,https://muckrack.com/zuzana-strakova-wenzlova-1
231729,231729,2020-08-18,https://muckrack.com/zvi-solomons
231730,231730,2019-08-17,https://muckrack.com/zviad-pochkhua


In [3]:
def get_top(n):
    temp = news[['author', 'article_count']].groupby('author').count().sort_values(['article_count'], ascending = False)
    temp = temp.reset_index().head(n)
    urls = []
    for i in range(len(temp)):
        url = re.sub(r'(\w*.\w*@\w*.\w*)', '', temp.loc[i].author.lower()) # remove emails
        url = re.sub(r'\|.*$', '', url) # remove everything after |
        url = re.sub(r'\,.*$', '', url) # remove evrything after ,
        url = re.sub(r'[^\w\s]', '', url) # remove punctuation
        url = ' '.join([i.strip() for i in url.split()]) # remove spaces and lowercase
        url = re.sub(r'(sa)(\s+)(\w+)(\s+)', '', url) # SA editors
        url = re.sub(r'(tulsa)(\s+)(world)', '', url) # tulsa world editors
        url = re.sub(r'(\d+)', '', url) # remove numbers
        url = 'https://muckrack.com/' + url.replace(' ', '-').strip('-') # make URL
        # print(muckrack[muckrack.url.str.contains(url, regex=False)])
        urls.append(url)
    temp['Muckrack'] = urls
    print(urls)
    return temp

get_top(300)

['https://muckrack.com/neil-shaw', 'https://muckrack.com/james-rodger', 'https://muckrack.com/jack-davis', 'https://muckrack.com/adam-wells', 'https://muckrack.com/sophie-mccoid', 'https://muckrack.com/joseph-zucker', 'https://muckrack.com/jack-otway', 'https://muckrack.com/simon-duke', 'https://muckrack.com/maren-estrada', 'https://muckrack.com/dave-johnson', 'https://muckrack.com/brett-gibbons', 'https://muckrack.com/kipp-jones', 'https://muckrack.com/jenna-ciccotelli', 'https://muckrack.com/rianne-addo', 'https://muckrack.com/scott-polacek', 'https://muckrack.com/clark-schultz', 'https://muckrack.com/lewis-winter', 'https://muckrack.com/rob-quinn', 'https://muckrack.com/tom-bull', 'https://muckrack.com/hannah-bleau', 'https://muckrack.com/annie-martin', 'https://muckrack.com/pam-key', 'https://muckrack.com/max-demara', 'https://muckrack.com/roxy-simons', 'https://muckrack.com/daniel-smith', 'https://muckrack.com/carl-surran', 'https://muckrack.com/daniel-blackham', 'https://muckrack

,author,article_count,Muckrack
0,Neil Shaw,523,https://muckrack.com/neil-shaw
1,James Rodger,512,https://muckrack.com/james-rodger
2,Jack Davis,251,https://muckrack.com/jack-davis
3,Adam Wells,223,https://muckrack.com/adam-wells
4,Sophie McCoid,218,https://muckrack.com/sophie-mccoid
...,...,...,...
295,Luke Kenton,77,https://muckrack.com/luke-kenton
296,Peter Sblendorio,77,https://muckrack.com/peter-sblendorio
297,Emily Hodgkin,77,https://muckrack.com/emily-hodgkin
298,Tom Towers,76,https://muckrack.com/tom-towers


In [ ]:
J.R. De Groote	
https://muckrack.com/unzela-khan
https://muckrack.com/jessica-dolcourt
https://muckrack.com/joel-b.-pollak wihtout . works
https://muckrack.com/liam-doyle, https://muckrack.com/liam-james-doyle works
https://muckrack.com/joseph-s.-pete-joseph.pete@nwi.com,-219-933-3316, https://muckrack.com/joseph-s-pete
https://muckrack.com/sean-o'grady, https://muckrack.com/sean-ogrady works
https://muckrack.com/mike--chiari, https://muckrack.com/mike-chiari
https://muckrack.com/ryan-taylor, https://muckrack.com/ryantaylorsport
https://muckrack.com/alessandra-scotto-di-santolo, https://muckrack.com/ali-malito

In [ ]:
input = {
  "query": "covid-19 situation atlanta, GA"
}
client = Algorithmia.client('simKLMHlWDMzWd+QcNRqVhI104r1')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=300) # optional
print(algo.pipe(input).result)

In [43]:
url = "https://muckrack.com/neil-shaw"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
sauce = urlopen(req).read()
soup = bs.BeautifulSoup(sauce,'lxml')

In [44]:
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VwcDWFVACwQIUVFQ",licenseKey:"8539532b2b",applicationID:"378502"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var i,o=t("ee"),a=t(23),c={};try{i=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(c.console=!0,i.indexOf("dev")!==-1&&(c.dev=!0),i.indexOf("nr_dev")!==-1&&(c.nrDev=!0))}catch(s){}c.nrDev&&o.on("internal-error",function(t){r(t.stack)}),c.dev&&o.on("fn-err",function(t,e,n){r(n.stack)}),c.dev&&(r("NR AGENT IN D

In [ ]:
if driver.find_element_by_class_name('js-as-seen-in-more'):
    driver.find_element_by_class_name('js-as-seen-in-more').click()
    time.sleep(1)
    print("clicked")

In [4]:
driver = webdriver.Chrome("chromedriver_win32\chromedriver.exe")

In [ ]:
https://muckrack.com/neil-shaw-1 # legit
https://muckrack.com/izzie-deibe # almost empty

In [37]:
driver.get("https://muckrack.com/neil-shaw-1")
driver.get("https://muckrack.com/adam-wells")
#driver.get("https://muckrack.com/izzie-deibe")
info = {}
if driver.current_url == 'https://muckrack.com/person/unavailable':
    print("Could not find page")
else:
#     if driver.find_elements_by_class_name('js-as-seen-in-more'):
#         driver.find_elements_by_class_name('js-as-seen-in-more')[0].click()
#         time.sleep(1)
        
#     if driver.find_elements_by_class_name('profile-verified'): 
#         info['verified'] = True
    
#     if driver.find_elements_by_xpath('//i[@class="fa fa-fw fa-building icon-standard"]'):
#         info['position'] = driver.find_elements_by_xpath('//i[@class="fa fa-fw fa-building icon-standard"]/following-sibling::div')[0].text
    
#     if driver.find_elements_by_xpath('//i[@class="fa fa-fw fa-map-marker icon-standard"]'):
#         info['located'] = driver.find_elements_by_xpath('//i[@class="fa fa-fw fa-map-marker icon-standard"]/following-sibling::div')[0].text
    
#     info['seen_in'] = list(set([i.text for i in driver.find_elements_by_tag_name('a') if '/media-outlet/' in i.get_attribute('href')]))
    
#     if driver.find_elements_by_xpath('//div[@class="mr-font-size-lg mr-font-family-2 top-sm"]'):
#         info['description'] = driver.find_elements_by_xpath('//div[@class="mr-font-size-lg mr-font-family-2 top-sm"]')[0].text
    
    if driver.find_elements_by_xpath('//div[@class="profile-section-social"]/a[@data-original-title="Website"]'):
        print(driver.find_elements_by_xpath('//div[@class="profile-section-social"]/a[@data-original-title="Website"]')[0].get_attribute('href'))
    
print(info)

{}


In [42]:
for i in driver.find_elements_by_xpath('//div[@class="mr-profile-section-content"]/div[@class="profile-section-social"]/*'):
    print(i.get_attribute('href'), i.get_attribute('data-original-title'))

https://bleacherreport.com/users/340764 None
http://twitter.com/adamwells1985 None
https://www.linkedin.com/in/adam-wells-07a756ab/ None
https://bleacherreport.com/users/340764 None
http://twitter.com/adamwells1985 None
https://www.linkedin.com/in/adam-wells-07a756ab/ None
